# Modelo

In [1]:
from classes import Network, MulticastGroup
from multicastpacking import MulticastPacking, solver
import reader

#instancen of the problem
file = "../../../MPP_instances/n30/b30_1.brite"

In [2]:
links = reader.get_network (file)
net = Network (links, nodes = 30)


mgroups = [MulticastGroup (g) for g in reader.get_groups (file) ]

problem = MulticastPacking (net, mgroups)

In [3]:
KSIZE = len(problem.groups)+1
NODES = net.nodes+1

## Criando a variável:  $x_{ij}^{kd}$

In [4]:
from gurobipy import *

m = Model ("MPP by Cost")

var_x = {}
var_y = {}

for link in net.links:
    for k in xrange (1, KSIZE):
        for d in problem.groups[k-1].members:
            x=link[0],link[1],k,d,
            y=link[1],link[0],k,d,
            var_x[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
            var_x[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
            
m.update ()



## Criando variável $y_{ij}^{k}$

In [5]:
for k in xrange (1, KSIZE):
    for link in net.links:
        x=link[0],link[1],k
        y=link[1],link[0],k
        var_y[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
        var_y[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

m.update ()

## Criando flow1 

In [6]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow1',k,d
        m.addConstr (
            quicksum ( var_x[x] for x in tuplelist (var_x).select ('*',sk,k,d) )
            -
            quicksum ( var_x[x] for x in tuplelist (var_x).select (sk,'*',k,d) )
            == -1,
            name=str(_name)
        )

m.update ()

## Criando flow2

In [7]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for j in xrange(1,NODES):
            sk = problem.groups[k-1].source
            _name='flow2',k,d,j,       
            m.addConstr (
                quicksum(
                   var_x[x] for x in tuplelist (var_x).select ('*',j,k,d) 
                    if x[1] not in [sk, d]
                )
                -
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select (j,'*',k,d) 
                    if x[0] not in [sk, d]
                )
                == 0,
                name=str(_name)
            )            

m.update ()

## Criando flow3

In [8]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow3',k,d
        m.addConstr (
            quicksum (
                var_x[x] for x in tuplelist (var_x).select ('*',d,k,d)
            )
            -
            quicksum (
                var_x[x] for x in tuplelist (var_x).select (d,'*',k,d)
            )
            == 1,
            name=str(_name)
        )
        
m.update ()

## Forçando $y_{ij}^{k}$ igual a 1 quando a aresta $(i,j)$ for usada

In [9]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for link in net.links:
            x=link[0],link[1],k,d,
            y=link[0],link[1],k
            _name='r4',link[0],link[1],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )            
            x=link[1],link[0],k,d,
            y=link[1],link[0],k
            _name='r4',link[1],link[0],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )
m.update ()

## Objective Function

In [10]:
expr  = []
for k in xrange (1, KSIZE ):
    expr.append ( quicksum (var_y[ (l[0],l[1],k) ] * net.links[l][0] 
        for l in net.links.keys()) )    
    expr.append ( quicksum (var_y[ (l[1],l[0],k) ] * net.links[l][0] 
        for l in net.links.keys()) )


m.setObjective (quicksum (expr), GRB.MINIMIZE)
m.update ()


## Forçando otimização sob restrição de capacidade

In [11]:
print (m)

<gurobi.Model MIP instance MPP by Cost: 4864 constrs, 4440 vars, Parameter changes: LogFile=gurobi.log>


In [12]:
m.write ('teste.lp')

In [13]:
m.optimize ()

Optimize a model with 4864 rows, 4440 columns and 15360 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [5e+01, 9e+02]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 77909
Presolve removed 124 rows and 64 columns
Presolve time: 0.04s
Presolved: 4740 rows, 4376 columns, 15160 nonzeros
Variable types: 0 continuous, 4376 integer (4376 binary)

Root relaxation: objective 1.364100e+04, 1343 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    13641.000000 13641.0000  0.00%     -    0s

Explored 0 nodes (1343 simplex iterations) in 0.08 seconds
Thread count was 2 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 1.364100000000e+04, best bound 1.364100000000e+04, gap 0.0%
